In [ ]:
#NEwspaper documentation: https://newspaper.readthedocs.io/en/latest/

In [204]:
from tqdm import tqdm
import newspaper
from newspaper import Article
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import re
from datetime import date
from matplotlib.pyplot import figure

In [2]:
today = date.today().day

In [46]:
#Looping for 14  days to get website object for last 2 weeks
website = []
for i in tqdm(range(today-14,today)):
    temp = newspaper.build("https://www.prnewswire.com/news-releases/financial-services-latest-news/investment-opinions-list/page=1&pagesize=100&month=11&day=" + str(i) + "&year=2021&hour=00",memoize_articles = False)
    website.append(temp)

100%|██████████| 14/14 [01:27<00:00,  6.22s/it]


In [48]:
#Try expect because some artices return 404 error, which stops the entire code
articles = []
for day in website:
    for i in tqdm(range(len(day.articles))):
        try:
            article = day.articles[i]
            article.download()
            article.parse()
            articles.append(article)
        except:
            pass

100%|██████████| 204/204 [01:32<00:00,  2.20it/s]


In [49]:
#Get the title,id,keyword,text and publishing date from each article
article_title = []
article_text = []
article_keywords = []
article_url = []
artical_publish_date = []
for article in tqdm(articles):
    article.nlp()
    artical_publish_date.append(article.publish_date)
    article_keywords.append(article.keywords)
    article_title.append(article.title)
    article_text.append(article.text)
    article_url.append(article.url)

100%|██████████| 2673/2673 [00:07<00:00, 346.91it/s]


In [114]:
#Store all the data into a dataframe
data = pd.DataFrame({"Article_Title": article_title,"Article_id": articles,"Keyword": article_keywords,"Article_Text": article_text,"Publish Date": artical_publish_date,"Url":article_url})

In [123]:
data

,Article_Title,Article_id,Keyword,Article_Text,Publish Date,Url
0,Internet of Things News and Press Releases fro...,<newspaper.article.Article object at 0x0000021...,"[press, subject, sources, request, submit, pro...","Journalists and Bloggers\n\nThe news you need,...",None,https://www.prnewswire.com/news-releases/consu...
1,All Heavy Industry & Manufacturing News and Pr...,<newspaper.article.Article object at 0x0000021...,"[press, manufacturing, subject, sources, reque...","Journalists and Bloggers\n\nThe news you need,...",None,https://www.prnewswire.com/news-releases/heavy...
2,"All Machine Tools, Metalworking and Metallurgy...",<newspaper.article.Article object at 0x0000021...,"[press, subject, sources, request, machine, su...","Journalists and Bloggers\n\nThe news you need,...",None,https://www.prnewswire.com/news-releases/heavy...
3,All Consumer Products & Retail News and Press ...,<newspaper.article.Article object at 0x0000021...,"[press, retail, subject, sources, request, sub...","Journalists and Bloggers\n\nThe news you need,...",None,https://www.prnewswire.com/news-releases/consu...
4,All Amusement Parks and Tourist Attractions Ne...,<newspaper.article.Article object at 0x0000021...,"[press, subject, sources, request, parks, subm...","Journalists and Bloggers\n\nThe news you need,...",None,https://www.prnewswire.com/news-releases/trave...
...,...,...,...,...,...,...
2668,What's the new normal for PR pros and press re...,<newspaper.article.Article object at 0x0000021...,"[press, release, cisions, normal, pr, releases...",What's the new normal for PR pros and press re...,2021-09-13 00:00:00,https://prnewswire.mediaroom.com/2021-09-13-Wh...
2669,"PR Newswire, Cision Communications Cloud rank ...",<newspaper.article.Article object at 0x0000021...,"[g2, ranked, reviews, pr, cloud, best, accordi...","PR Newswire, Cision Communications Cloud rank ...",2021-07-07 00:00:00,https://prnewswire.mediaroom.com/2021-07-07-PR...
2670,"PR Newswire, Cision Communications Cloud rank ...",<newspaper.article.Article object at 0x0000021...,"[g2, ranked, reviews, pr, cloud, best, accordi...","PR Newswire, Cision Communications Cloud rank ...",2021-07-07 00:00:00,https://prnewswire.mediaroom.com/2021-07-07-PR...
2671,Cision Announces Premium Sponsorship of the NI...,<newspaper.article.Article object at 0x0000021...,"[virtual, social, technology, solutions, commu...",Cision Announces Premium Sponsorship of the NI...,2021-06-21 00:00:00,https://prnewswire.mediaroom.com/2021-06-21-Ci...


In [124]:
#Dropping duplicates if any
data.drop_duplicates(subset = "Article_Text",inplace = True,ignore_index = True)

In [285]:
tickers = set()
for i in range(len(data)):
    temp = re.findall(r':\s[A-Z]{1,5}[)]',data.iloc[i]["Article_Text"])
    for tick in temp: 
        tickers.add(tick[-(len(tick) - 2):-1])

In [297]:
stocks = {}
for tick in tickers:
    stocks[tick] = yf.Ticker(tick).history(period = "YTD")

- RI: No data found for this date range, symbol may be delisted
